<a href="https://colab.research.google.com/github/nattaran/health-tequity-case-nasrin/blob/main/VoicePipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount Google Drive**

In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


**Set Working Directory**

In [7]:
import os

# Update this path to match your Google Drive folder
BASE_DIR = '/content/drive/MyDrive/health-tequity-case'
os.chdir(BASE_DIR)

# Verify
print(f"✅ Current directory: {os.getcwd()}")
print(f"📁 Files in directory: {os.listdir('.')}")

✅ Current directory: /content/drive/MyDrive/health-tequity-case
📁 Files in directory: ['Input_Audio_Files', 'Output_Audio_Files', 'Data', 'transcriptions_only.gsheet', 'whisper_transcriptions_with_errors.csv', 'transcriptions_only.csv', 'error_rates_summary.csv', 'data']


**install Required Packages**














In [8]:
# Install packages
!pip install -q openai-whisper
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q librosa soundfile
!pip install -q deep-translator
!pip install -q gtts
!pip install -q jiwer

print("✅ All packages installed successfully!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 13.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.6 MB/

**Import Libraries **

In [9]:
import whisper
import os
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
from deep_translator import GoogleTranslator
from gtts import gTTS
import jiwer
from jiwer import wer, cer
from jiwer import process_words
import warnings
import torch
warnings.filterwarnings('ignore')


print("✅ All libraries imported successfully!")

# Update this path to match your Google Drive folder


✅ All libraries imported successfully!


**Accessing to the Spanish Audio Files **

In [10]:
# AUDIO_FOLDER = '/content/drive/MyDrive/health-tequity-case/Input_Audio_Files/'  # UPDATE THIS

# # List all WAV files in the folder
# audio_files = sorted([f for f in os.listdir(AUDIO_FOLDER) if f.endswith('.wav')])

# print(f"📁 Found {len(audio_files)} WAV file(s):")
# for filename in audio_files:
#     print(f"  - {filename}")

📁 Found 4 WAV file(s):
  - q1_es.wav
  - q2_es.wav
  - q3_es.wav
  - q4_es.wav


In [ ]:
# GROUND_TRUTH_SPANISH = {
#     'q1_es.wav': '¿Cuáles son mis presiones sistólica y diastólica hoy?',
#     'q2_es.wav': '¿Cuáles fueron los valores durante la última semana?',
#     'q3_es.wav': '¿Cuál es la tendencia de los valores?',
#     'q4_es.wav': '¿Cuáles son los rangos normales para una persona como yo?'
# }

# # Ground truth English translations
# # UPDATE THESE with your actual ground truth translations
# GROUND_TRUTH_ENGLISH = {
#     'question1.wav': 'What are my systolic and diastolic blood pressures today?',
#     'question2.wav': 'What were the values over the last week?',
#     'question3.wav': 'What is the trend of the values?',
#     'question4.wav': 'What are the normal ranges for a person like me?'
# }

# print("\n✅ Ground truth loaded for error rate calculations")


✅ Ground truth loaded for error rate calculations


In [ ]:
#  Load Whisper Model
# def load_whisper_model(model_size="medium"):
#     """
#     Load Whisper model for ASR

#     Args:
#         model_size: 'tiny', 'base', 'small', 'medium', 'large'
#                    'medium' recommended for Spanish (good accuracy/speed balance)

#     Returns:
#         Loaded Whisper model
#     """
#     print(f"🤖 Loading Whisper '{model_size}' model...")
#     device = "cuda" if torch.cuda.is_available() else "cpu"
#     print(f"   Device: {device}")

#     model = whisper.load_model(model_size, device=device)
#     print("✅ Model loaded successfully!")
#     return model

# # Load the model
# model = load_whisper_model(model_size="medium")

🤖 Loading Whisper 'medium' model...
   Device: cpu
✅ Model loaded successfully!


# **Transcription Spanish Audio **

In [ ]:
# def transcribe_spanish_audio(model, audio_path, task="transcribe"):
#     """
#     Transcribe Spanish audio file using Whisper

#     Args:
#         model: Loaded Whisper model
#         audio_path: Path to WAV file
#         task: 'transcribe' (Spanish->Spanish) or 'translate' (Spanish->English)

#     Returns:
#         Dictionary with transcription results
#     """
#     print(f"🎵 Processing: {audio_path}")

#     result = model.transcribe(
#         audio_path,
#         language="spanish",  # Specify Spanish for better accuracy
#         task=task,
#         verbose=False
#     )

#     return result


# **Process All Audio Files**

In [ ]:
# def process_audio_files(model, audio_folder, audio_files):
#     """
#     Process multiple Spanish audio files from Google Drive

#     Args:
#         model: Loaded Whisper model
#         audio_folder: Path to folder containing audio files
#         audio_files: List of audio filenames

#     Returns:
#         DataFrame with transcription results
#     """
#     results = []

#     print("\n" + "="*60)
#     print("🎯 TRANSCRIBING SPANISH AUDIO FILES")
#     print("="*60)

#     for i, audio_file in enumerate(audio_files, 1):
#         audio_path = os.path.join(audio_folder, audio_file)
#         print(f"\n[{i}/{len(audio_files)}] {audio_file}")

#         if not os.path.exists(audio_path):
#             print(f"⚠️  Warning: {audio_path} not found, skipping...")
#             continue

#         # Get Spanish transcription
#         spanish_result = transcribe_spanish_audio(model, audio_path, task="transcribe")

#         # Get English translation
#         english_result = transcribe_spanish_audio(model, audio_path, task="translate")

#         results.append({
#             'question_number': i,
#             'audio_file': audio_file,
#             'spanish_transcription': spanish_result['text'].strip(),
#             'english_translation': english_result['text'].strip(),
#             'language_detected': spanish_result['language']
#         })

#         print(f"   🇪🇸 Spanish: {spanish_result['text'].strip()}")
#         print(f"   🇬🇧 English: {english_result['text'].strip()}")

#     return pd.DataFrame(results)

# # Process all files from Google Drive
# results_df = process_audio_files(model, AUDIO_FOLDER, audio_files)


🎯 TRANSCRIBING SPANISH AUDIO FILES

[1/4] q1_es.wav
🎵 Processing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q1_es.wav


100%|██████████| 324/324 [00:29<00:00, 11.15frames/s]


🎵 Processing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q1_es.wav


100%|██████████| 324/324 [00:27<00:00, 11.99frames/s]


   🇪🇸 Spanish: ¿Cuáles son mis valores de presión arterial hoy?
   🇬🇧 English: What are my blood pressure values today?

[2/4] q2_es.wav
🎵 Processing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q2_es.wav


...

   🇪🇸 Spanish: ¿Cuál es la tendencia de mis valores?
   🇬🇧 English: What is the trend of my values?

[4/4] q4_es.wav
🎵 Processing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q4_es.wav


100%|██████████| 398/398 [00:29<00:00, 13.44frames/s]


🎵 Processing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q4_es.wav


100%|██████████| 398/398 [00:28<00:00, 14.21frames/s]

   🇪🇸 Spanish: ¿Cuáles son los rangos normales para una persona como yo?
   🇬🇧 English: What are the normal ranges for a person like me?


** Error Rate Calculation Functions**

In [ ]:
# def calculate_wer(reference, hypothesis):
#     """
#     Calculate Word Error Rate (WER)
#     WER = (Substitutions + Deletions + Insertions) / Total Words in Reference

#     Args:
#         reference: Ground truth text
#         hypothesis: Predicted/transcribed text

#     Returns:
#         WER score (0-1), detailed metrics
#     """
#     # Normalize text (lowercase, strip)
#     ref = reference.lower().strip()
#     hyp = hypothesis.lower().strip()

#     # Calculate WER
#     wer_score = wer(ref, hyp)

#     # Get detailed measures
#     output= process_words(ref, hyp)

#     return {
#         'wer': wer_score,
#         'substitutions': output.substitutions,
#         'deletions': output.deletions,
#         'insertions': output.insertions,
#         'hits': output.hits
#     }

# def calculate_cer(reference, hypothesis):
#     """
#     Calculate Character Error Rate (CER)
#     CER = (Character Substitutions + Deletions + Insertions) / Total Characters

#     Args:
#         reference: Ground truth text
#         hypothesis: Predicted/transcribed text

#     Returns:
#         CER score (0-1)
#     """
#     ref = reference.lower().strip()
#     hyp = hypothesis.lower().strip()

#     cer_score = cer(ref, hyp)
#     return cer_score

# def calculate_ser(reference, hypothesis):
#     """
#     Calculate Sentence Error Rate (SER)
#     SER = 1 if sentences don't match exactly, 0 if they match

#     Args:
#         reference: Ground truth text
#         hypothesis: Predicted/transcribed text

#     Returns:
#         SER score (0 or 1)
#     """
#     # Normalize for comparison
#     ref = reference.lower().strip()
#     hyp = hypothesis.lower().strip()

#     # SER is 0 if exact match, 1 if not
#     ser_score = 0 if ref == hyp else 1
#     return ser_score

# def calculate_all_error_rates(results_df, ground_truth_spanish):
#     """
#     Calculate all error rates for Spanish and English transcriptions

#     Args:
#         results_df: DataFrame with transcription results
#         ground_truth_spanish: Dictionary of ground truth Spanish text
#         ground_truth_english: Dictionary of ground truth English text

#     Returns:
#         DataFrame with error rates
#     """
#     error_rates = []

#     for idx, row in results_df.iterrows():
#         audio_file = row['audio_file']

#         # Spanish error rates (ASR accuracy)
#         if audio_file in ground_truth_spanish:
#             spanish_gt = ground_truth_spanish[audio_file]
#             spanish_hyp = row['spanish_transcription']

#             spanish_wer_details = calculate_wer(spanish_gt, spanish_hyp)
#             spanish_cer_score = calculate_cer(spanish_gt, spanish_hyp)
#             spanish_ser_score = calculate_ser(spanish_gt, spanish_hyp)
#         else:
#             spanish_wer_details = {'wer': None, 'substitutions': None, 'deletions': None, 'insertions': None, 'hits': None}
#             spanish_cer_score = None
#             spanish_ser_score = None

#         # English error rates (Translation accuracy)
#         # if audio_file in ground_truth_english:
#         #     english_gt = ground_truth_english[audio_file]
#         #     english_hyp = row['english_translation']

#         #     english_wer_details = calculate_wer(english_gt, english_hyp)
#         #     english_cer_score = calculate_cer(english_gt, english_hyp)
#         #     english_ser_score = calculate_ser(english_gt, english_hyp)
#         # else:
#         #     english_wer_details = {'wer': None, 'substitutions': None, 'deletions': None, 'insertions': None, 'hits': None}
#         #     english_cer_score = None
#         #     english_ser_score = None

#         error_rates.append({
#             'question_number': row['question_number'],
#             'audio_file': audio_file,

#             # Spanish metrics
#             'spanish_wer': spanish_wer_details['wer'],
#             'spanish_substitutions': spanish_wer_details['substitutions'],
#             'spanish_deletions': spanish_wer_details['deletions'],
#             'spanish_insertions': spanish_wer_details['insertions'],
#             'spanish_cer': spanish_cer_score,
#             'spanish_ser': spanish_ser_score,

#             # English metrics
#             # 'english_wer': english_wer_details['wer'],
#             # 'english_substitutions': english_wer_details['substitutions'],
#             # 'english_deletions': english_wer_details['deletions'],
#             # 'english_insertions': english_wer_details['insertions'],
#             # 'english_cer': english_cer_score,
#             # 'english_ser': english_ser_score
#         })

#     return pd.DataFrame(error_rates)

# # Calculate error rates
# print("\n" + "="*60)
# print("📊 CALCULATING ERROR RATES")
# print("="*60)

# error_rates_df = calculate_all_error_rates(results_df, GROUND_TRUTH_SPANISH)

# # Merge with original results
# full_results_df = results_df.merge(error_rates_df, on=['question_number', 'audio_file'])


📊 CALCULATING ERROR RATES


Display Results with Error Rates

In [ ]:
# ============================================
# CELL 7: Display Results with Error Rates
# ============================================
# display_cols = ['question_number', 'audio_file', 'spanish_wer', 'spanish_cer',
#                 'spanish_ser']  #, 'english_wer', 'english_cer', 'english_ser']
# print(full_results_df[display_cols].to_string(index=False))

# # Calculate averages (skip None values)
# print("\n" + "="*60)
# print("📈 AVERAGE ERROR RATES")
# print("="*60)

# # Spanish metrics
# spanish_wer_mean = full_results_df['spanish_wer'].dropna().mean()
# spanish_cer_mean = full_results_df['spanish_cer'].dropna().mean()
# spanish_ser_mean = full_results_df['spanish_ser'].dropna().mean()

# print(f"Spanish ASR:")
# if not pd.isna(spanish_wer_mean):
#     print(f"  Average WER: {spanish_wer_mean:.4f} ({spanish_wer_mean*100:.2f}%)")
#     print(f"  Average CER: {spanish_cer_mean:.4f} ({spanish_cer_mean*100:.2f}%)")
#     print(f"  Average SER: {spanish_ser_mean:.4f} ({spanish_ser_mean*100:.2f}%)")
# else:
#     print(f"  ⚠️  No ground truth available for Spanish ASR")

# # English metrics
# # english_wer_mean = full_results_df['english_wer'].dropna().mean()
# # english_cer_mean = full_results_df['english_cer'].dropna().mean()
# # english_ser_mean = full_results_df['english_ser'].dropna().mean()

# # print(f"\nEnglish Translation:")
# # if not pd.isna(english_wer_mean):
# #     print(f"  Average WER: {english_wer_mean:.4f} ({english_wer_mean*100:.2f}%)")
# #     print(f"  Average CER: {english_cer_mean:.4f} ({english_cer_mean*100:.2f}%)")
# #     print(f"  Average SER: {english_ser_mean:.4f} ({english_ser_mean*100:.2f}%)")
# # else:
# #     print(f"  ⚠️  No ground truth available for English translation")

# # ============================================
# # SAVE RESULTS
# # ============================================
# print("\n" + "="*60)
# print("💾 SAVING RESULTS")
# print("="*60)

# # Save full results with all columns
# output_file_full = "whisper_transcriptions_with_errors.csv"
# full_results_df.to_csv(output_file_full, index=False)
# print(f"✅ Full results saved to: {output_file_full}")

# # Save transcriptions only (simplified)
# transcriptions_only = full_results_df[['question_number', 'audio_file',
#                                         'spanish_transcription', 'english_translation']].copy()
# output_file_transcriptions = "transcriptions_only.csv"
# transcriptions_only.to_csv(output_file_transcriptions, index=False)
# print(f"✅ Transcriptions only saved to: {output_file_transcriptions}")

# # Save error rates summary
# error_summary = full_results_df[['question_number', 'audio_file',
#                                   'spanish_wer', 'spanish_cer', 'spanish_ser']].copy()
# output_file_errors = "error_rates_summary.csv"
# error_summary.to_csv(output_file_errors, index=False)
# print(f"✅ Error rates summary saved to: {output_file_errors}")

# # Save to Google Drive (if mounted)
# try:
#     # Try to save to Drive if it's mounted
#     drive_output_folder = '/content/drive/MyDrive/health_tequity_results/'
#     os.makedirs(drive_output_folder, exist_ok=True)

#     full_results_df.to_csv(os.path.join(drive_output_folder, output_file_full), index=False)
#     transcriptions_only.to_csv(os.path.join(drive_output_folder, output_file_transcriptions), index=False)
#     error_summary.to_csv(os.path.join(drive_output_folder, output_file_errors), index=False)

#     print(f"\n✅ All files also saved to Google Drive: {drive_output_folder}")
# except:
#     print("\n⚠️  Google Drive not mounted - files saved locally only")

# # Download files to your computer
# print("\n📥 Download files to your computer:")
# from google.colab import files
# files.download(output_file_full)
# files.download(output_file_transcriptions)
# files.download(output_file_errors)

# print("\n✅ All results saved successfully!")

 question_number audio_file  spanish_wer  spanish_cer  spanish_ser
               1  q1_es.wav     0.500000     0.452830            1
               2  q2_es.wav     0.125000     0.096154            1
               3  q3_es.wav     0.142857     0.054054            1
               4  q4_es.wav     0.000000     0.000000            0

📈 AVERAGE ERROR RATES
Spanish ASR:
  Average WER: 0.1920 (19.20%)
  Average CER: 0.1508 (15.08%)
  Average SER: 0.7500 (75.00%)

💾 SAVING RESULTS
✅ Full results saved to: whisper_transcriptions_with_errors.csv
✅ Transcriptions only saved to: transcriptions_only.csv
✅ Error rates summary saved to: error_rates_summary.csv

✅ All files also saved to Google Drive: /content/drive/MyDrive/health_tequity_results/

📥 Download files to your computer:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ All results saved successfully!


Save Results

Display Individual Results with Comparision

In [ ]:
# print("\n" + "="*60)
# print("📝 DETAILED TRANSCRIPTIONS WITH GROUND TRUTH COMPARISON")
# print("="*60)

# for idx, row in full_results_df.iterrows():
#     print(f"\n{'='*60}")
#     print(f"Question {row['question_number']}: {row['audio_file']}")
#     print(f"{'='*60}")

#     # Spanish comparison
#     print(f"\n🇪🇸 SPANISH:")
#     audio_file = row['audio_file']
#     if audio_file in GROUND_TRUTH_SPANISH:
#         print(f"  Ground Truth: {GROUND_TRUTH_SPANISH[audio_file]}")
#         print(f"  Transcribed:  {row['spanish_transcription']}")

#         # Display metrics if available
#         if row['spanish_wer'] is not None:
#             print(f"  WER: {row['spanish_wer']:.4f} | CER: {row['spanish_cer']:.4f} | SER: {row['spanish_ser']}")
#             print(f"  Errors: Subs={row['spanish_substitutions']}, Dels={row['spanish_deletions']}, Ins={row['spanish_insertions']}")
#         else:
#             print(f"  ⚠️  No ground truth available for error calculation")
#     else:
#         print(f"  Transcribed:  {row['spanish_transcription']}")
#         print(f"  ⚠️  No ground truth available")

#     # # English comparison
#     # print(f"\n🇬🇧 ENGLISH:")
#     # if audio_file in GROUND_TRUTH_ENGLISH:
#     #     print(f"  Ground Truth: {GROUND_TRUTH_ENGLISH[audio_file]}")
#     #     print(f"  Translated:   {row['english_translation']}")

#     #     # Display metrics if available
#     #     if row['english_wer'] is not None:
#     #         print(f"  WER: {row['english_wer']:.4f} | CER: {row['english_cer']:.4f} | SER: {row['english_ser']}")
#     #         print(f"  Errors: Subs={row['english_substitutions']}, Dels={row['english_deletions']}, Ins={row['english_insertions']}")
#     #     else:
#     #         print(f"  ⚠️  No ground truth available for error calculation")
#     # else:
#     #     print(f"  Translated:   {row['english_translation']}")
#     #     print(f"  ⚠️  No ground truth available")
#     print()


📝 DETAILED TRANSCRIPTIONS WITH GROUND TRUTH COMPARISON

Question 1: q1_es.wav

🇪🇸 SPANISH:
  Ground Truth: ¿Cuáles son mis presiones sistólica y diastólica hoy?
  Transcribed:  ¿Cuáles son mis valores de presión arterial hoy?
  WER: 0.5000 | CER: 0.4528 | SER: 1
  Errors: Subs=4, Dels=0, Ins=0


Question 2: q2_es.wav

🇪🇸 SPANISH:
  Ground Truth: ¿Cuáles fueron los valores durante la última semana?
  Transcribed:  ¿Cuáles fueron los valores de la última semana?
  WER: 0.1250 | CER: 0.0962 | SER: 1
  Errors: Subs=1, Dels=0, Ins=0


Question 3: q3_es.wav

🇪🇸 SPANISH:
  Ground Truth: ¿Cuál es la tendencia de los valores?
  Transcribed:  ¿Cuál es la tendencia de mis valores?
  WER: 0.1429 | CER: 0.0541 | SER: 1
  Errors: Subs=1, Dels=0, Ins=0


Question 4: q4_es.wav

🇪🇸 SPANISH:
  Ground Truth: ¿Cuáles son los rangos normales para una persona como yo?
  Transcribed:  ¿Cuáles son los rangos normales para una persona como yo?
  WER: 0.0000 | CER: 0.0000 | SER: 0
  Errors: Subs=0, Dels=0, Ins

LLM **part**

In [13]:
!pip -q install transformers accelerate bitsandbytes sentencepiece einops safetensors



Traceback (most recent call last):
object address  : 0x7cc33a10a680
object refcount : 2
object type     : 0x9d7580
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
^C


In [11]:
# Optional: see what Colab gave you
!nvidia-smi || true

import sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())

/bin/bash: line 1: nvidia-smi: command not found
Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
Platform: Linux-6.6.105+-x86_64-with-glibc2.35


In [14]:
!pip -q install "openai==1.51.2" "httpx==0.27.2" "httpcore==1.0.5" pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
firebase-admin 6.9.0 requires httpx[http2]==0.28.1, but you have httpx 0.27.2 which is incompatible.
google-genai 1.25.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


In [15]:
from google.colab import userdata
api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("Add OPENAI_API_KEY in the Secrets panel (left sidebar, key icon).")

In [16]:
# Upload the CSV from google drive

from google.colab import drive
drive.mount('/content/drive')
csv_path = "/content/drive/MyDrive/health-tequity-case/Data/synthetic_bp_one_person.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
# Load CSV as plain text
# csv_text = open(CSV_PATH, "r", encoding="utf-8").read()
# print("\n".join(csv_text.splitlines()[:5]))



In [6]:
# pip install --upgrade openai


In [15]:
# ======================================
# STEP: Answer questions + Translate + Save
# ======================================
# import json, re, pandas as pd, os
# from openai import OpenAI

# # Initialize client
# # client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
# client = OpenAI(api_key=api_key)
# # ==========================
# # Templates and System Prompt (same as before)
# # ==========================
# TEMPLATES = {
#     "today":         "Your systolic blood pressure was {sys} mm of Hg and your diastolic blood pressure was {dia} mm of Hg.",
#     "last_week":     "Over the last week, your systolic blood pressure has averaged {sys_avg} mm of Hg and your diastolic blood pressure has averaged {dia_avg} mm of Hg.",
#     "trend_month":   "The trend for the values over the last month has been {trend} average values of your systolic blood pressure and diastolic blood pressure.",
#     "normal_ranges": "While each person’s normal range should be discussed with their physician, literature suggests that for a {sex} aged {age} years, systolic and diastolic blood pressure can typically be expected to be {sys_norm} mm Hg and {dia_norm} mm Hg respectively. This information was retrieved from {reference}."
# }

# SYSTEM = """You are a careful data analyst.
# Do ALL analysis yourself using ONLY the CSV provided by the user.
# Interpret columns: date, age, sex, systolic, diastolic.
# - "Today" = most recent row by date.
# - "Last week" = last 7 rows by date (including the most recent).
# - "Trend over the last month" = last 30 rows; return one of: increasing / decreasing / stable.
# - If a specific date is mentioned (e.g., 'on October 1'), return that date’s values if present.
# Return STRICT JSON ONLY:
# {
#  "template": "today"|"last_week"|"trend_month"|"normal_ranges",
#  "fields": {...},
#  "final_text": "one sentence exactly following the chosen template with mm of Hg units"
# }
# No extra prose. JSON only.
# """

# # ==========================
# # Core Function: Ask GPT for Answer (English)
# # ==========================
# def ask_gpt(question_en: str, csv_block: str) -> dict:
#     """Queries GPT for a structured English answer from CSV."""
#     user = f"CSV:\n{csv_block}\n\nQUESTION:\n{question_en}\n\nReturn JSON only."
#     resp = client.chat.completions.create(
#         model="gpt-4o",
#         temperature=0,
#         messages=[
#             {"role": "system", "content": SYSTEM},
#             {"role": "system", "content": "Templates:\n" + json.dumps(TEMPLATES)},
#             {"role": "user", "content": user}
#         ]
#     ).choices[0].message.content

#     # Clean response
#     clean = re.sub(r"^```json|```$", "", resp.strip(), flags=re.M | re.I)
#     start, end = clean.find("{"), clean.rfind("}")
#     return json.loads(clean[start:end+1])


# # ==========================
# # Translation Function (English → Spanish)
# # ==========================
# def translate_to_spanish(english_text: str) -> str:
#     """Uses GPT-4 to translate an English medical answer into Spanish."""
#     prompt = f"""
# Translate the following English text into clear, neutral Spanish suitable for a healthcare context.
# Do NOT add commentary or explanations.

# Text:
# {english_text}
# """
#     trans = client.responses.create(model="gpt-4o-mini", input=prompt)
#     return trans.output_text.strip()


# # ==========================
# # Run the Full Pipeline
# # ==========================
# def run_pipeline(csv_path: str):
#     df = pd.read_csv(csv_path)
#     csv_block = df.head(50).to_csv(index=False)  # send limited rows for context

#     questions = {
#         "q1": "What are my systolic and diastolic blood pressures today?",
#         "q2": "What were the values over the last week?",
#         "q3": "What is the trend of the values?",
#         "q4": "What are the normal ranges for a person like me?",
#         "q5": "what is my systolic blood pressure on October 10?"
#     }

#     results = []
#     for qid, qtext in questions.items():
#         print(f"\n🔹 {qid}: {qtext}")
#         try:
#             ans = ask_gpt(qtext, csv_block)
#             english = ans.get("final_text", "")
#             spanish = translate_to_spanish(english)
#             results.append({
#                 "question_id": qid,
#                 "question_en": qtext,
#                 "answer_en": english,
#                 "answer_es": spanish,
#                 "template_used": ans.get("template", ""),
#                 "fields": json.dumps(ans.get("fields", {}))
#             })
#             print(f"✅ Completed {qid}")
#         except Exception as e:
#             print(f"❌ Error in {qid}: {e}")

#     out_df = pd.DataFrame(results)
#     os.makedirs("data", exist_ok=True)
#     out_df.to_csv("/content/drive/MyDrive/health-tequity-case/Data/answers_en_es.csv", index=False)
#     print("\n✅ All results saved to data/answers_en_es.csv")
#     return out_df


# # ==========================
# # Execute
# # ==========================
# results_df = run_pipeline("/content/drive/MyDrive/health-tequity-case/Data/synthetic_bp_one_person.csv")
# display(results_df)


In [4]:
# ============================================
# Generalized LLM Q&A + Translation Pipeline
# ============================================

# import json, re, os, pandas as pd
# from openai import OpenAI

# # Initialize OpenAI client (make sure your .env has OPENAI_API_KEY)
# client = OpenAI(api_key=api_key)
# # -------------------------------
# # 1. Updated System Prompt (Flexible)
# # -------------------------------
# SYSTEM = """
# You are a careful data analyst.
# You receive a synthetic blood pressure dataset with columns: date, age, sex, systolic, diastolic.

# Do ALL analysis yourself using ONLY the CSV data provided by the user.
# You may be asked about daily values, weekly averages, monthly trends, minimum/maximum readings, or comparisons by sex or age.

# When you answer:
# - Perform all calculations directly from the CSV.
# - Use precise dates or time windows when mentioned.
# - Always include numeric values (in mm Hg).
# - Keep your response factual and concise (1–2 sentences).

# Return JSON only:
# {
#  "answer": "<your full English sentence answer>",
#  "computed_fields": { "any intermediate numeric results used" }
# }
# """

# # -------------------------------
# # 2. Ask GPT Function
# # -------------------------------
# def ask_gpt(question_en: str, csv_block: str) -> dict:
#     """
#     Ask GPT-4 to answer any blood pressure–related question using the CSV data.
#     Returns a dict with 'answer' and 'computed_fields'.
#     """
#     user = f"CSV data:\n{csv_block}\n\nQUESTION:\n{question_en}\n\nReturn JSON only as specified."

#     response = client.chat.completions.create(
#         model="gpt-4o",  # or gpt-4o-mini for smaller cost
#         temperature=0,
#         messages=[
#             {"role": "system", "content": SYSTEM},
#             {"role": "user", "content": user}
#         ]
#     ).choices[0].message.content

#     # Clean the LLM output for JSON safety
#     clean = re.sub(r"^```json|```$", "", response.strip(), flags=re.M | re.I)
#     start, end = clean.find("{"), clean.rfind("}")
#     return json.loads(clean[start:end+1])


# # -------------------------------
# # 3. Translation Function (English → Spanish)
# # -------------------------------
# def translate_to_spanish(english_text: str) -> str:
#     """
#     Uses GPT-4 to translate an English medical answer into clear, neutral Spanish.
#     """
#     prompt = f"""
# Translate the following English medical text into clear, neutral Spanish suitable for a healthcare context.
# Do NOT add commentary or explanations.

# Text:
# {english_text}
# """
#     trans = client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=[{"role": "user", "content": prompt}]
#     ).choices[0].message.content.strip()

#     return trans


# # -------------------------------
# # 4. Main Pipeline Function
# # -------------------------------
# def run_pipeline(csv_path: str, questions: list[str]):
#     """
#     Runs the full LLM + translation pipeline:
#     - Reads the dataset
#     - Answers each question in English
#     - Translates each answer to Spanish
#     - Saves results to CSV
#     """
#     df = pd.read_csv(csv_path)
#     # Send a subset or the full CSV as text
#     csv_block = df.head(50).to_csv(index=False)

#     results = []
#     for i, q in enumerate(questions, 1):
#         print(f"\n🔹 Question {i}: {q}")
#         try:
#             ans = ask_gpt(q, csv_block)
#             english_answer = ans.get("answer", "").strip()
#             spanish_answer = translate_to_spanish(english_answer)
#             results.append({
#                 "question": q,
#                 "answer_en": english_answer,
#                 "answer_es": spanish_answer,
#                 "computed_fields": json.dumps(ans.get("computed_fields", {}))
#             })
#             print(f"✅ Completed Question {i}")
#         except Exception as e:
#             print(f"❌ Error in Question {i}: {e}")

#     # Save bilingual answers
#     os.makedirs("data", exist_ok=True)
#     out_df = pd.DataFrame(results)
#     out_path = "/content/drive/MyDrive/health-tequity-case/Data/answers_en_es_october17.csv"
#     out_df.to_csv(out_path, index=False)
#     print(f"\n✅ All results saved to {out_path}")
#     return out_df


# # -------------------------------
# # 5. Run the Pipeline
# # -------------------------------
# # You can pass any question list here
# questions = [
#     "What are my systolic and diastolic blood pressures today?",
#     "What was my average blood pressure over the last two weeks?",
#     "When was my highest systolic blood pressure recorded?",
#     "How has my diastolic pressure changed over the last 10 days?",
#     "What was my blood pressure on October 5th?",
#     "Compare my average systolic and diastolic pressures for the first and last week."
# ]

# results_df = run_pipeline("/content/drive/MyDrive/health-tequity-case/Data/synthetic_bp_one_person.csv", questions)
# display(results_df)



In [3]:
# ============================================
# Generalized LLM Q&A + Translation + TTS Pipeline
# ============================================

# import json, re, os, pandas as pd
# from openai import OpenAI

# # Initialize OpenAI client
# client = OpenAI(api_key=api_key)

# # -------------------------------
# # 1. Updated System Prompt (Flexible)
# # -------------------------------
# SYSTEM = """
# You are a careful data analyst.
# You receive a synthetic blood pressure dataset with columns: date, age, sex, systolic, diastolic.

# Do ALL analysis yourself using ONLY the CSV data provided by the user.
# You may be asked about daily values, weekly averages, monthly trends, minimum/maximum readings, or comparisons by sex or age.

# When you answer:
# - Perform all calculations directly from the CSV.
# - Use precise dates or time windows when mentioned.
# - Always include numeric values (in mm Hg).
# - Keep your response factual and concise (1–2 sentences).

# Return JSON only:
# {
#  "answer": "<your full English sentence answer>",
#  "computed_fields": { "any intermediate numeric results used" }
# }
# """

# # -------------------------------
# # 2. Ask GPT Function
# # -------------------------------
# def ask_gpt(question_en: str, csv_block: str) -> dict:
#     """
#     Ask GPT-4 to answer any blood pressure–related question using the CSV data.
#     Returns a dict with 'answer' and 'computed_fields'.
#     """
#     user = f"CSV data:\n{csv_block}\n\nQUESTION:\n{question_en}\n\nReturn JSON only as specified."

#     response = client.chat.completions.create(
#         model="gpt-4o",
#         temperature=0,
#         messages=[
#             {"role": "system", "content": SYSTEM},
#             {"role": "user", "content": user}
#         ]
#     ).choices[0].message.content

#     clean = re.sub(r"^```json|```$", "", response.strip(), flags=re.M | re.I)
#     start, end = clean.find("{"), clean.rfind("}")
#     return json.loads(clean[start:end+1])

# # -------------------------------
# # 3. Translation Function (English → Spanish)
# # -------------------------------
# def translate_to_spanish(english_text: str) -> str:
#     """
#     Uses GPT-4 to translate an English medical answer into clear, neutral Spanish.
#     """
#     prompt = f"""
# Translate the following English medical text into clear, neutral Spanish suitable for a healthcare context.
# Do NOT add commentary or explanations.

# Text:
# {english_text}
# """
#     trans = client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=[{"role": "user", "content": prompt}]
#     ).choices[0].message.content.strip()

#     return trans

# # -------------------------------
# # 4. Text-to-Speech Function (Spanish → Audio)
# # -------------------------------
# def text_to_speech_spanish(text: str, filename: str, voice: str = "alloy"):
#     """
#     Convert Spanish text to spoken audio (.wav) using OpenAI TTS.
#     """
#     try:
#         os.makedirs(os.path.dirname(filename), exist_ok=True)
#         with client.audio.speech.with_streaming_response.create(
#             model="gpt-4o-mini-tts",
#             voice=voice,
#             input=text
#         ) as response:
#             response.stream_to_file(filename)
#         print(f"🔊 Saved Spanish audio: {filename}")
#         return filename
#     except Exception as e:
#         print(f"❌ TTS generation failed for '{filename}': {e}")
#         return None

# # -------------------------------
# # 5. Main Pipeline Function
# # -------------------------------
# def run_pipeline(csv_path: str, questions: list[str]):
#     """
#     Runs the full LLM + translation + TTS pipeline:
#     - Reads the dataset
#     - Answers each question in English
#     - Translates each answer to Spanish
#     - Converts Spanish text to Spanish audio
#     - Saves all outputs to CSV
#     """
#     df = pd.read_csv(csv_path)
#     csv_block = df.to_csv(index=False)

#     results = []
#     os.makedirs("data/audio", exist_ok=True)

#     for i, q in enumerate(questions, 1):
#         print(f"\n🔹 Question {i}: {q}")
#         try:
#             ans = ask_gpt(q, csv_block)
#             english_answer = ans.get("answer", "").strip()
#             spanish_answer = translate_to_spanish(english_answer)

#             # Generate Spanish TTS
#             audio_filename = f"/content/drive/MyDrive/health-tequity-case/Output_Audio_Files/answer_{i}_es.wav"
#             text_to_speech_spanish(spanish_answer, audio_filename)

#             results.append({
#                 "question": q,
#                 "answer_en": english_answer,
#                 "answer_es": spanish_answer,
#                 "audio_file": audio_filename,
#                 "computed_fields": json.dumps(ans.get("computed_fields", {}))
#             })
#             print(f"✅ Completed Question {i}")

#         except Exception as e:
#             print(f"❌ Error in Question {i}: {e}")

#     # Save all results
#     out_df = pd.DataFrame(results)
#     out_path = "/content/drive/MyDrive/health-tequity-case/Data/answers_with_audio_october17.csv"
#     out_df.to_csv(out_path, index=False)
#     print(f"\n✅ All results (with audio) saved to {out_path}")
#     return out_df

# # -------------------------------
# # 6. Run the Pipeline
# # -------------------------------
# questions = [
#     "What are my systolic and diastolic blood pressures today?",
#     "What was my average blood pressure over the last two weeks?",
#     "When was my highest systolic blood pressure recorded?",
#     "How has my diastolic pressure changed over the last 10 days?",
#     "What was my blood pressure on October 5th?",
#     "Compare my average systolic and diastolic pressures for the first and last week."
# ]

# results_df = run_pipeline("/content/drive/MyDrive/health-tequity-case/Data/synthetic_bp_one_person.csv", questions)
# display(results_df)


# **This is the complete pipeline - it has all different steps**

In [15]:
# ============================================
# HEALTH TEQUITY CASE – COMPLETE PIPELINE
# Audio → Transcription → LLM → Translation → TTS
# ============================================

import os, json, re, pandas as pd
import whisper
from openai import OpenAI

# ================================================================
# 🔧 0️⃣ CONFIGURATION: Define Folder Paths
# ================================================================
BASE_PATH = "/content/drive/MyDrive/health-tequity-case"

# Input folder (Spanish audio questions)
AUDIO_INPUT_FOLDER = os.path.join(BASE_PATH, "Input_Audio_Files")

# Output folder for generated Spanish answers (audio)
AUDIO_OUTPUT_FOLDER = os.path.join(BASE_PATH, "data", "audio_out")

# Folder for CSV results (transcriptions + final pipeline output)
CSV_OUTPUT_FOLDER = os.path.join(BASE_PATH, "data", "csv_results")

# Make sure output folders exist
os.makedirs(AUDIO_OUTPUT_FOLDER, exist_ok=True)
os.makedirs(CSV_OUTPUT_FOLDER, exist_ok=True)

# Check input folder validity
if not os.path.exists(AUDIO_INPUT_FOLDER):
    raise FileNotFoundError(f"❌ Input folder not found: {AUDIO_INPUT_FOLDER}")

audio_files = [f for f in os.listdir(AUDIO_INPUT_FOLDER) if f.lower().endswith(('.wav', '.mp3', '.m4a'))]
if not audio_files:
    raise ValueError(f"❌ No audio files found in {AUDIO_INPUT_FOLDER}. Please add Spanish question files.")

print(f"✅ Using input folder: {AUDIO_INPUT_FOLDER}")
print(f"✅ Found {len(audio_files)} audio file(s): {audio_files}")
print(f"📁 Output audio: {AUDIO_OUTPUT_FOLDER}")
print(f"📁 CSV results: {CSV_OUTPUT_FOLDER}")

# Initialize OpenAI client
client = OpenAI(api_key=api_key)

# ================================================================
# 1️⃣ AUDIO → SPANISH TRANSCRIPTION → ENGLISH TRANSLATION
# ================================================================
def transcribe_spanish_audio(model, audio_path):
    """Transcribe Spanish audio using Whisper."""
    print(f"🎧 Transcribing: {audio_path}")
    result = model.transcribe(audio_path, language="spanish", task="transcribe", verbose=False)
    return result["text"].strip(), result["language"]

def translate_spanish_to_english(spanish_text: str) -> str:
    """Translate Spanish transcription into English using GPT-4."""
    prompt = f"""
Translate the following Spanish medical question into clear English.
Keep it accurate and concise.

Text:
{spanish_text}
"""
    result = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return result.choices[0].message.content.strip()

def process_and_translate_audio(audio_folder: str, audio_files: list[str], output_csv: str):
    """
    Transcribe & translate all Spanish audio questions and save CSV.
    """
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)
    model = whisper.load_model("base")
    all_results = []

    print("\n" + "="*60)
    print("🎯 STARTING SPANISH TRANSCRIPTION + TRANSLATION")
    print("="*60)

    for i, audio_file in enumerate(audio_files, 1):
        audio_path = os.path.join(audio_folder, audio_file)
        spanish_text, detected_lang = transcribe_spanish_audio(model, audio_path)
        english_text = translate_spanish_to_english(spanish_text)

        all_results.append({
            "question_number": i,
            "audio_file": audio_file,
            "spanish_transcription": spanish_text,
            "english_translation": english_text,
            "language_detected": detected_lang
        })

        print(f"\n[{i}] {audio_file}")
        print(f"🇪🇸 {spanish_text}")
        print(f"🇬🇧 {english_text}")

    df = pd.DataFrame(all_results)
    df.to_csv(output_csv, index=False)
    print(f"\n✅ Transcriptions and translations saved to {output_csv}")
    return df

# ================================================================
# 2️⃣ GPT ANALYSIS + TRANSLATION + TTS
# ================================================================
SYSTEM = """
You are a careful data analyst.
You receive a synthetic blood pressure dataset with columns: date, age, sex, systolic, diastolic.
Do ALL analysis yourself using ONLY the CSV data provided by the user.
You may be asked about daily values, weekly averages, monthly trends, min/max readings, or comparisons by sex or age.
Return JSON only:
{ "answer": "<English text>", "computed_fields": { "intermediate results" } }
"""

def ask_gpt(question_en: str, csv_block: str) -> dict:
    """Ask GPT-4 to analyze blood pressure data using an English question."""
    user = f"CSV data:\n{csv_block}\n\nQUESTION:\n{question_en}"
    resp = client.chat.completions.create(
        model="gpt-4o",
        temperature=0,
        messages=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": user}
        ]
    ).choices[0].message.content
    clean = re.sub(r"^```json|```$", "", resp.strip(), flags=re.M | re.I)
    start, end = clean.find("{"), clean.rfind("}")
    return json.loads(clean[start:end+1])

def translate_to_spanish(english_text: str) -> str:
    """Translate English medical text to Spanish."""
    prompt = f"Translate this English medical text into clear, neutral Spanish:\n{english_text}"
    return client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    ).choices[0].message.content.strip()

def text_to_speech_spanish(text: str, filename: str, voice="alloy"):
    """Convert Spanish text to speech (TTS)."""
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with client.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts", voice=voice, input=text
    ) as response:
        response.stream_to_file(filename)
    print(f"🔊 Saved Spanish audio: {filename}")
    return filename

# ================================================================
# 3️⃣ MAIN PIPELINE
# ================================================================
def run_full_pipeline(csv_path: str, audio_folder: str, audio_files: list[str]):
    """
    Full workflow:
      1. Transcribe + translate Spanish audio questions
      2. Use English translation for GPT-4 analysis
      3. Translate answers to Spanish
      4. Convert Spanish answers to speech
      5. Save all results to CSV
    """
    # Step 1 – Transcription + translation
    trans_csv = os.path.join(CSV_OUTPUT_FOLDER, "audio_translations.csv")
    trans_df = process_and_translate_audio(audio_folder, audio_files, trans_csv)

    # Step 2 – Load BP dataset
    df_bp = pd.read_csv(csv_path)
    csv_block = df_bp.to_csv(index=False)

    results = []
    for i, row in trans_df.iterrows():
        q_num, q_en = row["question_number"], row["english_translation"]
        print(f"\n🔹 Q{q_num}: {q_en}")

        try:
            # Step 3 – GPT Analysis
            ans = ask_gpt(q_en, csv_block)
            ans_en = ans.get("answer", "").strip()

            # Step 4 – Translate answer → Spanish
            ans_es = translate_to_spanish(ans_en)

            # Step 5 – Convert answer → audio
            audio_file = os.path.join(AUDIO_OUTPUT_FOLDER, f"answer_{q_num}_es.wav")
            text_to_speech_spanish(ans_es, audio_file)

            results.append({
                "question_number": q_num,
                "audio_file_in": row["audio_file"],
                "spanish_question": row["spanish_transcription"],
                "english_question": q_en,
                "english_answer": ans_en,
                "spanish_answer": ans_es,
                "audio_answer_file": audio_file,
                "computed_fields": json.dumps(ans.get("computed_fields", {}))
            })

            print(f"✅ Completed Q{q_num}")

        except Exception as e:
            print(f"❌ Error for Q{q_num}: {e}")

    # Step 6 – Save final combined results
    final_csv = os.path.join(CSV_OUTPUT_FOLDER, "final_pipeline_results.csv")
    out_df = pd.DataFrame(results)
    out_df.to_csv(final_csv, index=False)
    print(f"\n✅ All results (audio + translations + analysis) saved to {final_csv}")
    return out_df

# ================================================================
# 4️⃣ RUN PIPELINE
# ================================================================
csv_path = os.path.join(BASE_PATH, "Data", "synthetic_bp_one_person.csv")

results_df = run_full_pipeline(csv_path, AUDIO_INPUT_FOLDER, audio_files)
display(results_df)



✅ Using input folder: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files
✅ Found 4 audio file(s): ['q2_es.wav', 'q3_es.wav', 'q4_es.wav', 'q1_es.wav']
📁 Output audio: /content/drive/MyDrive/health-tequity-case/data/audio_out
📁 CSV results: /content/drive/MyDrive/health-tequity-case/data/csv_results


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 301MiB/s]



🎯 STARTING SPANISH TRANSCRIPTION + TRANSLATION
🎧 Transcribing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q2_es.wav


100%|██████████| 312/312 [00:03<00:00, 98.69frames/s]



[1] q2_es.wav
🇪🇸 ¿Cuáles fueron los valores de la última semana?
🇬🇧 What were the values from the last week?
🎧 Transcribing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q3_es.wav


...

100%|██████████| 398/398 [00:02<00:00, 141.06frames/s]



[3] q4_es.wav
🇪🇸 ¿Cuáles son los rango normales para una persona como yo?
🇬🇧 What are the normal ranges for someone like me?
🎧 Transcribing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q1_es.wav


100%|██████████| 324/324 [00:02<00:00, 138.20frames/s]



[4] q1_es.wav
🇪🇸 ¿Cuáles son mis valores de presión arterial hoy?
🇬🇧 What are my blood pressure values today?

✅ Transcriptions and translations saved to /content/drive/MyDrive/health-tequity-case/data/csv_results/audio_translations.csv

🔹 Q1: What were the values from the last week?
🔊 Saved Spanish audio: /content/drive/MyDrive/health-tequity-case/data/audio_out/answer_1_es.wav
✅ Completed Q1

🔹 Q2: What is the trend of my values?
🔊 Saved Spanish audio: /content/drive/MyDrive/health-tequity-case/data/audio_out/answer_2_es.wav
✅ Completed Q2

🔹 Q3: What are the normal ranges for someone like me?
🔊 Saved Spanish audio: /content/drive/MyDrive/health-tequity-case/data/audio_out/answer_3_es.wav
✅ Completed Q3

🔹 Q4: What are my blood pressure values today?
🔊 Saved Spanish audio: /content/drive/MyDrive/health-tequity-case/data/audio_out/answer_4_es.wav
✅ Completed Q4

✅ All results (audio + translations + analysis) saved to /content/drive/MyDrive/health-tequity-case/data/csv_results/final_

,question_number,audio_file_in,spanish_question,english_question,english_answer,spanish_answer,audio_answer_file,computed_fields
0,1,q2_es.wav,¿Cuáles fueron los valores de la última semana?,What were the values from the last week?,The blood pressure readings for the last week ...,Las lecturas de presión arterial de la última ...,/content/drive/MyDrive/health-tequity-case/dat...,"{""2025-10-10"": {""systolic"": 160, ""diastolic"": ..."
1,2,q3_es.wav,¿Cuál es la tendencia de mis valores?,What is the trend of my values?,The blood pressure readings show a fluctuating...,Las lecturas de presión arterial muestran una ...,/content/drive/MyDrive/health-tequity-case/dat...,"{""daily_readings"": {""2025-09-17"": {""systolic"":..."
2,3,q4_es.wav,¿Cuáles son los rango normales para una person...,What are the normal ranges for someone like me?,"For a 68-year-old female, normal blood pressur...","Para una mujer de 68 años, los rangos normales...",/content/drive/MyDrive/health-tequity-case/dat...,"{""normal_systolic_range"": ""<120 mmHg"", ""normal..."
3,4,q1_es.wav,¿Cuáles son mis valores de presión arterial hoy?,What are my blood pressure values today?,"Your blood pressure values today, on 2025-10-1...","Los valores de su presión arterial hoy, 16 de ...",/content/drive/MyDrive/health-tequity-case/dat...,"{""today_date"": ""2025-10-16"", ""today_systolic"":..."


In [27]:
# this pipeline has the WER /CER and sentence error rate computation

# ================================================================
# HEALTH TEQUITY CASE – COMPLETE PIPELINE
# Audio → Transcription → WER → LLM → Translation → TTS
# ================================================================

import os, json, re, pandas as pd
import whisper
from openai import OpenAI
from jiwer import wer, mer, wil
from jiwer import process_words
import Levenshtein

# ================================================================
# 0️⃣ CONFIGURATION
# ================================================================
BASE_PATH = "/content/drive/MyDrive/health-tequity-case"

# Define key folders
AUDIO_INPUT_FOLDER = os.path.join(BASE_PATH, "Input_Audio_Files")
AUDIO_OUTPUT_FOLDER = os.path.join(BASE_PATH, "data", "audio_out")
CSV_OUTPUT_FOLDER = os.path.join(BASE_PATH, "data", "csv_results")

# Ensure output folders exist
os.makedirs(AUDIO_OUTPUT_FOLDER, exist_ok=True)
os.makedirs(CSV_OUTPUT_FOLDER, exist_ok=True)

# Check input folder
if not os.path.exists(AUDIO_INPUT_FOLDER):
    raise FileNotFoundError(f"❌ Input folder not found: {AUDIO_INPUT_FOLDER}")

audio_files = [f for f in os.listdir(AUDIO_INPUT_FOLDER) if f.lower().endswith(('.wav', '.mp3', '.m4a'))]
if not audio_files:
    raise ValueError(f"❌ No audio files found in {AUDIO_INPUT_FOLDER}")

print(f"✅ Found {len(audio_files)} audio file(s): {audio_files}")

# Initialize OpenAI client
client = OpenAI(api_key=api_key)


# ================================================================
# 1️⃣ AUDIO → SPANISH TRANSCRIPTION + ENGLISH TRANSLATION
# ================================================================
def transcribe_spanish_audio(model, audio_path):
    print(f"🎧 Transcribing: {audio_path}")
    result = model.transcribe(audio_path, language="spanish", task="transcribe", verbose=False)
    return result["text"].strip(), result["language"]

def translate_spanish_to_english(spanish_text: str) -> str:
    """Translate Spanish transcription to English."""
    prompt = f"Translate the following Spanish medical question into clear English:\n\n{spanish_text}"
    result = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return result.choices[0].message.content.strip()

def process_and_translate_audio(audio_folder, audio_files, output_csv):
    model = whisper.load_model("base")
    all_results = []

    print("\n🎯 STARTING SPANISH TRANSCRIPTION + TRANSLATION\n" + "="*60)
    for i, audio_file in enumerate(audio_files, 1):
        audio_path = os.path.join(audio_folder, audio_file)
        if not os.path.exists(audio_path):
            print(f"⚠️ {audio_file} not found, skipping...")
            continue

        spanish_text, detected_lang = transcribe_spanish_audio(model, audio_path)
        english_text = translate_spanish_to_english(spanish_text)

        all_results.append({
            "audio_file": audio_file,
            "spanish_transcription": spanish_text,
            "english_translation": english_text,
            "language_detected": detected_lang
        })

        print(f"\n[{i}] {audio_file}")
        print(f"🇪🇸 {spanish_text}")
        print(f"🇬🇧 {english_text}")

    df = pd.DataFrame(all_results)
    df.to_csv(output_csv, index=False)
    print(f"\n✅ Transcriptions + translations saved to {output_csv}")
    return df


# ================================================================
# 2️⃣ ASR EVALUATION (WER, CER, SER)
# ================================================================

# ================================================================
# 🧮  ASR EVALUATION: Compute WER, CER, SER for Whisper Transcriptions
# ================================================================
from jiwer import wer, process_words
import Levenshtein

def compute_cer(reference: str, hypothesis: str) -> float:
    """
    Compute Character Error Rate (CER) using Levenshtein distance.
    CER = edit_distance / len(reference)
    """
    reference, hypothesis = reference.strip(), hypothesis.strip()
    if not reference:
        return 1.0 if hypothesis else 0.0
    return Levenshtein.distance(reference, hypothesis) / len(reference)

def compute_sentence_error(reference: str, hypothesis: str) -> int:
    """
    Compute Sentence Error Rate (SER): 0 if identical, 1 otherwise.
    """
    return 0 if reference.strip() == hypothesis.strip() else 1


def evaluate_asr_performance(
    ground_truth_csv: str,
    transcribed_csv: str,
    output_csv: str = os.path.join(CSV_OUTPUT_FOLDER, "asr_metrics.csv")
):
    """
    Compare Whisper-generated transcriptions with ground truth transcriptions.
    Compute WER, substitutions, deletions, insertions, CER, and SER.
    """
    if not os.path.exists(ground_truth_csv):
        raise FileNotFoundError(f"❌ Missing GT file: {ground_truth_csv}")
    if not os.path.exists(transcribed_csv):
        raise FileNotFoundError(f"❌ Missing transcription CSV: {transcribed_csv}")

    gt_df = pd.read_csv(ground_truth_csv)
    tr_df = pd.read_csv(transcribed_csv)
    gt_df.columns = [c.lower().strip() for c in gt_df.columns]
    tr_df.columns = [c.lower().strip() for c in tr_df.columns]

    df = pd.merge(gt_df, tr_df, on="audio_file", how="inner")
    print(f"\n🎯 Evaluating {len(df)} audio files for ASR performance...\n")

    results = []
    for i, row in df.iterrows():
        ref = str(row["ground_truth"])
        hyp = str(row["spanish_transcription"])

        # Get detailed WER components using new jiwer API
        measures = process_words(ref, hyp)
        wer_score = round(measures.wer, 4)
        subs = measures.substitutions
        dels = measures.deletions
        ins = measures.insertions

        # Compute CER & SER
        cer = round(compute_cer(ref, hyp), 4)
        ser = compute_sentence_error(ref, hyp)

        results.append({
            "audio_file": row["audio_file"],
            "ground_truth": ref,
            "whisper_transcription": hyp,
            "WER": wer_score,
            "Substitutions": subs,
            "Deletions": dels,
            "Insertions": ins,
            "CER": cer,
            "SER": ser
        })

        print(f"🎧 {row['audio_file']} → WER: {wer_score}, CER: {cer}, SER: {ser}")

    out_df = pd.DataFrame(results)
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)
    out_df.to_csv(output_csv, index=False)
    print(f"\n✅ ASR metrics saved to: {output_csv}")

    return out_df


# ================================================================
# 3️⃣ GPT DATA ANALYSIS + TRANSLATION + TTS
# ================================================================
SYSTEM = """
You are a careful data analyst.
You receive a synthetic blood pressure dataset with columns: date, age, sex, systolic, diastolic.
Do ALL analysis yourself using ONLY the CSV provided.
Answer questions like: daily readings, averages, trends, comparisons, etc.
Return JSON:
{ "answer": "<English answer>", "computed_fields": { "numeric values used" } }
"""

def ask_gpt(question_en, csv_block):
    user = f"CSV data:\n{csv_block}\n\nQUESTION:\n{question_en}"
    resp = client.chat.completions.create(
        model="gpt-4o",
        temperature=0,
        messages=[{"role": "system", "content": SYSTEM}, {"role": "user", "content": user}]
    ).choices[0].message.content
    clean = re.sub(r"^```json|```$", "", resp.strip(), flags=re.M | re.I)
    start, end = clean.find("{"), clean.rfind("}")
    return json.loads(clean[start:end+1])

def translate_to_spanish(english_text):
    prompt = f"Translate this English medical answer into clear, neutral Spanish:\n{english_text}"
    return client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    ).choices[0].message.content.strip()

def text_to_speech_spanish(text, filename, voice="alloy"):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with client.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts", voice=voice, input=text
    ) as response:
        response.stream_to_file(filename)
    print(f"🔊 Saved Spanish audio: {filename}")
    return filename


# ================================================================
# 4️⃣ MAIN PIPELINE
# ================================================================
def run_full_pipeline(csv_path, audio_folder, audio_files):
    # Step 1 — Transcribe and Translate Spanish Audio
    trans_csv = os.path.join(CSV_OUTPUT_FOLDER, "audio_translations.csv")
    trans_df = process_and_translate_audio(audio_folder, audio_files, trans_csv)

    # Step 2 — Evaluate ASR (WER, CER, SER)
    gt_csv = os.path.join(audio_folder, "ground_truth.csv")
    asr_csv = os.path.join(CSV_OUTPUT_FOLDER, "asr_metrics.csv")
    asr_df = evaluate_asr_performance(gt_csv, trans_csv, asr_csv)

    # Step 3 — Load Blood Pressure Data
    df_bp = pd.read_csv(csv_path)
    csv_block = df_bp.to_csv(index=False)

    results = []
    for i, row in trans_df.iterrows():
        q_num = i + 1
        q_en = row["english_translation"]
        print(f"\n🔹 Q{q_num}: {q_en}")

        try:
            ans = ask_gpt(q_en, csv_block)
            ans_en = ans.get("answer", "").strip()
            ans_es = translate_to_spanish(ans_en)

            audio_file = os.path.join(AUDIO_OUTPUT_FOLDER, f"answer_{q_num}_es.wav")
            text_to_speech_spanish(ans_es, audio_file)

            results.append({
                "question_number": q_num,
                "audio_file_in": row["audio_file"],
                "spanish_question": row["spanish_transcription"],
                "english_question": q_en,
                "english_answer": ans_en,
                "spanish_answer": ans_es,
                "audio_answer_file": audio_file,
                "computed_fields": json.dumps(ans.get("computed_fields", {}))
            })
            print(f"✅ Completed Q{q_num}")

        except Exception as e:
            print(f"❌ Error Q{q_num}: {e}")

    # Step 4 — Save Final Results
    final_csv = os.path.join(CSV_OUTPUT_FOLDER, "final_pipeline_results.csv")
    pd.DataFrame(results).to_csv(final_csv, index=False)
    print(f"\n✅ All results saved to {final_csv}")
    return results


# ================================================================
# 5️⃣ RUN
# ================================================================
csv_path = os.path.join(BASE_PATH, "Data", "synthetic_bp_one_person.csv")
run_full_pipeline(csv_path, AUDIO_INPUT_FOLDER, audio_files)






✅ Found 4 audio file(s): ['q2_es.wav', 'q3_es.wav', 'q4_es.wav', 'q1_es.wav']

🎯 STARTING SPANISH TRANSCRIPTION + TRANSLATION
🎧 Transcribing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q2_es.wav


100%|██████████| 312/312 [00:03<00:00, 85.86frames/s]



[1] q2_es.wav
🇪🇸 ¿Cuáles fueron los valores de la última semana?
🇬🇧 What were the values from the last week?
🎧 Transcribing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q3_es.wav


100%|██████████| 261/261 [00:02<00:00, 102.02frames/s]



[2] q3_es.wav
🇪🇸 ¿Cuál es la tendencia de mis valores?
🇬🇧 What is the trend of my values?
🎧 Transcribing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q4_es.wav


100%|██████████| 398/398 [00:03<00:00, 102.86frames/s]



[3] q4_es.wav
🇪🇸 ¿Cuáles son los rango normales para una persona como yo?
🇬🇧 What are the normal ranges for someone like me?
🎧 Transcribing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q1_es.wav


100%|██████████| 324/324 [00:02<00:00, 121.28frames/s]



[4] q1_es.wav
🇪🇸 ¿Cuáles son mis valores de presión arterial hoy?
🇬🇧 What are my blood pressure readings today?

✅ Transcriptions + translations saved to /content/drive/MyDrive/health-tequity-case/data/csv_results/audio_translations.csv

🎯 Evaluating 4 audio files for ASR performance...

🎧 q1_es.wav → WER: 0.0, CER: 0.0, SER: 0
🎧 q2_es.wav → WER: 0.0, CER: 0.0, SER: 0
🎧 q3_es.wav → WER: 0.0, CER: 0.0, SER: 0
🎧 q4_es.wav → WER: 0.1, CER: 0.0175, SER: 1

✅ ASR metrics saved to: /content/drive/MyDrive/health-tequity-case/data/csv_results/asr_metrics.csv

🔹 Q1: What were the values from the last week?
🔊 Saved Spanish audio: /content/drive/MyDrive/health-tequity-case/data/audio_out/answer_1_es.wav
✅ Completed Q1

🔹 Q2: What is the trend of my values?
🔊 Saved Spanish audio: /content/drive/MyDrive/health-tequity-case/data/audio_out/answer_2_es.wav
✅ Completed Q2

🔹 Q3: What are the normal ranges for someone like me?
🔊 Saved Spanish audio: /content/drive/MyDrive/health-tequity-case/data/audio

[{'question_number': 1,
  'audio_file_in': 'q2_es.wav',
  'spanish_question': '¿Cuáles fueron los valores de la última semana?',
  'english_question': 'What were the values from the last week?',
  'english_answer': 'The blood pressure readings for the last week (from 2025-10-10 to 2025-10-16) are as follows: \n- 2025-10-10: Systolic 160 mmHg, Diastolic 101 mmHg, Regime hypertensive, Category hypertensive\n- 2025-10-11: Systolic 152 mmHg, Diastolic 94 mmHg, Regime hypertensive, Category hypertensive\n- 2025-10-12: Systolic 157 mmHg, Diastolic 98 mmHg, Regime hypertensive, Category hypertensive\n- 2025-10-13: Systolic 144 mmHg, Diastolic 100 mmHg, Regime hypertensive, Category hypertensive\n- 2025-10-14: Systolic 145 mmHg, Diastolic 91 mmHg, Regime hypertensive, Category hypertensive\n- 2025-10-15: Systolic 124 mmHg, Diastolic 81 mmHg, Regime normal, Category elevated\n- 2025-10-16: Systolic 110 mmHg, Diastolic 76 mmHg, Regime normal, Category normal.',
  'spanish_answer': 'Las lecturas 

In [24]:
# # ============================================
# # HEALTH TEQUITY CASE – COMPLETE PIPELINE
# # Audio → Transcription → LLM → Translation → TTS
# # ============================================

# import os, json, re, pandas as pd
# import whisper
# from openai import OpenAI

# # ================================================================
# # 🔧 0️⃣ CONFIGURATION: Define Folder Paths
# # ================================================================
# BASE_PATH = "/content/drive/MyDrive/health-tequity-case"

# # Input folder (Spanish audio questions)
# AUDIO_INPUT_FOLDER = os.path.join(BASE_PATH, "Input_Audio_Files")

# # Output folder for generated Spanish answers (audio)
# AUDIO_OUTPUT_FOLDER = os.path.join(BASE_PATH, "data", "audio_out")

# # Folder for CSV results (transcriptions + final pipeline output)
# CSV_OUTPUT_FOLDER = os.path.join(BASE_PATH, "data", "csv_results")

# # Make sure output folders exist
# os.makedirs(AUDIO_OUTPUT_FOLDER, exist_ok=True)
# os.makedirs(CSV_OUTPUT_FOLDER, exist_ok=True)

# # Check input folder validity
# if not os.path.exists(AUDIO_INPUT_FOLDER):
#     raise FileNotFoundError(f"❌ Input folder not found: {AUDIO_INPUT_FOLDER}")

# audio_files = [f for f in os.listdir(AUDIO_INPUT_FOLDER) if f.lower().endswith(('.wav', '.mp3', '.m4a'))]
# if not audio_files:
#     raise ValueError(f"❌ No audio files found in {AUDIO_INPUT_FOLDER}. Please add Spanish question files.")

# print(f"✅ Using input folder: {AUDIO_INPUT_FOLDER}")
# print(f"✅ Found {len(audio_files)} audio file(s): {audio_files}")
# print(f"📁 Output audio: {AUDIO_OUTPUT_FOLDER}")
# print(f"📁 CSV results: {CSV_OUTPUT_FOLDER}")

# # Initialize OpenAI client
# client = OpenAI(api_key=api_key)

# # ================================================================
# # 1️⃣ AUDIO → SPANISH TRANSCRIPTION → ENGLISH TRANSLATION
# # ================================================================
# def transcribe_spanish_audio(model, audio_path):
#     """Transcribe Spanish audio using Whisper."""
#     print(f"🎧 Transcribing: {audio_path}")
#     result = model.transcribe(audio_path, language="spanish", task="transcribe", verbose=False)
#     return result["text"].strip(), result["language"]

# def translate_spanish_to_english(spanish_text: str) -> str:
#     """Translate Spanish transcription into English using GPT-4."""
#     prompt = f"""
# Translate the following Spanish medical question into clear English.
# Keep it accurate and concise.

# Text:
# {spanish_text}
# """
#     result = client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=[{"role": "user", "content": prompt}]
#     )
#     return result.choices[0].message.content.strip()

# def process_and_translate_audio(audio_folder: str, audio_files: list[str], output_csv: str):
#     """
#     Transcribe & translate all Spanish audio questions and save CSV.
#     """
#     os.makedirs(os.path.dirname(output_csv), exist_ok=True)
#     model = whisper.load_model("base")
#     all_results = []

#     print("\n" + "="*60)
#     print("🎯 STARTING SPANISH TRANSCRIPTION + TRANSLATION")
#     print("="*60)

#     for i, audio_file in enumerate(audio_files, 1):
#         audio_path = os.path.join(audio_folder, audio_file)
#         spanish_text, detected_lang = transcribe_spanish_audio(model, audio_path)
#         english_text = translate_spanish_to_english(spanish_text)

#         all_results.append({
#             "question_number": i,
#             "audio_file": audio_file,
#             "spanish_transcription": spanish_text,
#             "english_translation": english_text,
#             "language_detected": detected_lang
#         })

#         print(f"\n[{i}] {audio_file}")
#         print(f"🇪🇸 {spanish_text}")
#         print(f"🇬🇧 {english_text}")

#     df = pd.DataFrame(all_results)
#     df.to_csv(output_csv, index=False)
#     print(f"\n✅ Transcriptions and translations saved to {output_csv}")
#     return df

# # ================================================================
# # 2️⃣ GPT ANALYSIS + TRANSLATION + TTS
# # ================================================================
# SYSTEM = """
# You are a careful data analyst.
# You receive a synthetic blood pressure dataset with columns: date, age, sex, systolic, diastolic.
# Do ALL analysis yourself using ONLY the CSV data provided by the user.
# You may be asked about daily values, weekly averages, monthly trends, min/max readings, or comparisons by sex or age.
# Return JSON only:
# { "answer": "<English text>", "computed_fields": { "intermediate results" } }
# """

# def ask_gpt(question_en: str, csv_block: str) -> dict:
#     """Ask GPT-4 to analyze blood pressure data using an English question."""
#     user = f"CSV data:\n{csv_block}\n\nQUESTION:\n{question_en}"
#     resp = client.chat.completions.create(
#         model="gpt-4o",
#         temperature=0,
#         messages=[
#             {"role": "system", "content": SYSTEM},
#             {"role": "user", "content": user}
#         ]
#     ).choices[0].message.content
#     clean = re.sub(r"^```json|```$", "", resp.strip(), flags=re.M | re.I)
#     start, end = clean.find("{"), clean.rfind("}")
#     return json.loads(clean[start:end+1])

# def translate_to_spanish(english_text: str) -> str:
#     """Translate English medical text to Spanish."""
#     prompt = f"Translate this English medical text into clear, neutral Spanish:\n{english_text}"
#     return client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=[{"role": "user", "content": prompt}]
#     ).choices[0].message.content.strip()

# def text_to_speech_spanish(text: str, filename: str, voice="alloy"):
#     """Convert Spanish text to speech (TTS)."""
#     os.makedirs(os.path.dirname(filename), exist_ok=True)
#     with client.audio.speech.with_streaming_response.create(
#         model="gpt-4o-mini-tts", voice=voice, input=text
#     ) as response:
#         response.stream_to_file(filename)
#     print(f"🔊 Saved Spanish audio: {filename}")
#     return filename

# # ================================================================
# # 3️⃣ MAIN PIPELINE
# # ================================================================
# def run_full_pipeline(csv_path: str, audio_folder: str, audio_files: list[str]):
#     """
#     Full workflow:
#       1. Transcribe + translate Spanish audio questions
#       2. Use English translation for GPT-4 analysis
#       3. Translate answers to Spanish
#       4. Convert Spanish answers to speech
#       5. Save all results to CSV
#     """
#     # Step 1 – Transcription + translation
#     trans_csv = os.path.join(CSV_OUTPUT_FOLDER, "audio_translations.csv")
#     trans_df = process_and_translate_audio(audio_folder, audio_files, trans_csv)

#     # Step 2 – Load BP dataset
#     df_bp = pd.read_csv(csv_path)
#     csv_block = df_bp.to_csv(index=False)

#     results = []
#     for i, row in trans_df.iterrows():
#         q_num, q_en = row["question_number"], row["english_translation"]
#         print(f"\n🔹 Q{q_num}: {q_en}")

#         try:
#             # Step 3 – GPT Analysis
#             ans = ask_gpt(q_en, csv_block)
#             ans_en = ans.get("answer", "").strip()

#             # Step 4 – Translate answer → Spanish
#             ans_es = translate_to_spanish(ans_en)

#             # Step 5 – Convert answer → audio
#             audio_file = os.path.join(AUDIO_OUTPUT_FOLDER, f"answer_{q_num}_es.wav")
#             text_to_speech_spanish(ans_es, audio_file)

#             results.append({
#                 "question_number": q_num,
#                 "audio_file_in": row["audio_file"],
#                 "spanish_question": row["spanish_transcription"],
#                 "english_question": q_en,
#                 "english_answer": ans_en,
#                 "spanish_answer": ans_es,
#                 "audio_answer_file": audio_file,
#                 "computed_fields": json.dumps(ans.get("computed_fields", {}))
#             })

#             print(f"✅ Completed Q{q_num}")

#         except Exception as e:
#             print(f"❌ Error for Q{q_num}: {e}")

#     # Step 6 – Save final combined results
#     final_csv = os.path.join(CSV_OUTPUT_FOLDER, "final_pipeline_results.csv")
#     out_df = pd.DataFrame(results)
#     out_df.to_csv(final_csv, index=False)
#     print(f"\n✅ All results (audio + translations + analysis) saved to {final_csv}")
#     return out_df

# # ================================================================
# # 4️⃣ RUN PIPELINE
# # ================================================================
# csv_path = os.path.join(BASE_PATH, "Data", "synthetic_bp_one_person.csv")

# results_df = run_full_pipeline(csv_path, AUDIO_INPUT_FOLDER, audio_files)
# display(results_df)



In [ ]:
# this pipeline has the WER /CER and sentence error rate computation

# ================================================================
# HEALTH TEQUITY CASE – COMPLETE PIPELINE
# Audio → Transcription → WER → LLM → Translation → TTS
# ================================================================

import os, json, re, pandas as pd
import whisper
from openai import OpenAI
from jiwer import wer, mer, wil
from jiwer import process_words
import Levenshtein

# ================================================================
# 0️⃣ CONFIGURATION
# ================================================================
BASE_PATH = "/content/drive/MyDrive/health-tequity-case"

# Define key folders
AUDIO_INPUT_FOLDER = os.path.join(BASE_PATH, "Input_Audio_Files")
AUDIO_OUTPUT_FOLDER = os.path.join(BASE_PATH, "data", "audio_out")
CSV_OUTPUT_FOLDER = os.path.join(BASE_PATH, "data", "csv_results")

# Ensure output folders exist
os.makedirs(AUDIO_OUTPUT_FOLDER, exist_ok=True)
os.makedirs(CSV_OUTPUT_FOLDER, exist_ok=True)

# Check input folder
if not os.path.exists(AUDIO_INPUT_FOLDER):
    raise FileNotFoundError(f"❌ Input folder not found: {AUDIO_INPUT_FOLDER}")

audio_files = [f for f in os.listdir(AUDIO_INPUT_FOLDER) if f.lower().endswith(('.wav', '.mp3', '.m4a'))]
if not audio_files:
    raise ValueError(f"❌ No audio files found in {AUDIO_INPUT_FOLDER}")

print(f"✅ Found {len(audio_files)} audio file(s): {audio_files}")

# Initialize OpenAI client
client = OpenAI(api_key=api_key)


# ================================================================
# 1️⃣ AUDIO → SPANISH TRANSCRIPTION + ENGLISH TRANSLATION
# ================================================================
def transcribe_spanish_audio(model, audio_path):
    print(f"🎧 Transcribing: {audio_path}")
    result = model.transcribe(audio_path, language="spanish", task="transcribe", verbose=False)
    return result["text"].strip(), result["language"]

def translate_spanish_to_english(spanish_text: str) -> str:
    """Translate Spanish transcription to English."""
    prompt = f"Translate the following Spanish medical question into clear English:\n\n{spanish_text}"
    result = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return result.choices[0].message.content.strip()

def process_and_translate_audio(audio_folder, audio_files, output_csv):
    model = whisper.load_model("base")
    all_results = []

    print("\n🎯 STARTING SPANISH TRANSCRIPTION + TRANSLATION\n" + "="*60)
    for i, audio_file in enumerate(audio_files, 1):
        audio_path = os.path.join(audio_folder, audio_file)
        if not os.path.exists(audio_path):
            print(f"⚠️ {audio_file} not found, skipping...")
            continue

        spanish_text, detected_lang = transcribe_spanish_audio(model, audio_path)
        english_text = translate_spanish_to_english(spanish_text)

        all_results.append({
            "audio_file": audio_file,
            "spanish_transcription": spanish_text,
            "english_translation": english_text,
            "language_detected": detected_lang
        })

        print(f"\n[{i}] {audio_file}")
        print(f"🇪🇸 {spanish_text}")
        print(f"🇬🇧 {english_text}")

    df = pd.DataFrame(all_results)
    df.to_csv(output_csv, index=False)
    print(f"\n✅ Transcriptions + translations saved to {output_csv}")
    return df


# ================================================================
# 2️⃣ ASR EVALUATION (WER, CER, SER)
# ================================================================
def compute_cer(reference: str, hypothesis: str) -> float:
    """Character Error Rate (CER)."""
    reference, hypothesis = reference.strip(), hypothesis.strip()
    if not reference:
        return 1.0 if hypothesis else 0.0
    return Levenshtein.distance(reference, hypothesis) / len(reference)

def compute_sentence_error(reference: str, hypothesis: str) -> int:
    """Sentence Error Rate (SER)."""
    return 0 if reference.strip() == hypothesis.strip() else 1

def evaluate_asr_performance(ground_truth_csv, transcribed_csv, output_csv):
    if not os.path.exists(ground_truth_csv):
        raise FileNotFoundError(f"❌ Missing GT file: {ground_truth_csv}")
    if not os.path.exists(transcribed_csv):
        raise FileNotFoundError(f"❌ Missing transcription file: {transcribed_csv}")

    gt_df = pd.read_csv(ground_truth_csv)
    tr_df = pd.read_csv(transcribed_csv)

    gt_df.columns = [c.lower().strip() for c in gt_df.columns]
    tr_df.columns = [c.lower().strip() for c in tr_df.columns]

    df = pd.merge(gt_df, tr_df, on="audio_file", how="inner")

    print(f"\n🎯 Evaluating {len(df)} files for ASR metrics...\n")

    results = []
    for _, row in df.iterrows():
        ref = str(row["ground_truth"])
        hyp = str(row["spanish_transcription"])

        m = wer(ref, hyp)
        cer = compute_cer(ref, hyp)
        ser = compute_sentence_error(ref, hyp)

        results.append({
            "audio_file": row["audio_file"],
            "ground_truth": ref,
            "whisper_transcription": hyp,
            "WER": round(m["wer"], 4),
            "Substitutions": m["substitutions"],
            "Deletions": m["deletions"],
            "Insertions": m["insertions"],
            "CER": round(cer, 4),
            "SER": ser
        })

        print(f"🎧 {row['audio_file']} → WER={m['wer']:.3f}, CER={cer:.3f}, SER={ser}")

    out_df = pd.DataFrame(results)
    out_df.to_csv(output_csv, index=False)
    print(f"\n✅ ASR metrics saved to {output_csv}")
    return out_df


# ================================================================
# 3️⃣ GPT DATA ANALYSIS + TRANSLATION + TTS
# ================================================================
SYSTEM = """
You are a careful data analyst.
You receive a synthetic blood pressure dataset with columns: date, age, sex, systolic, diastolic.
Do ALL analysis yourself using ONLY the CSV provided.
Answer questions like: daily readings, averages, trends, comparisons, etc.
Return JSON:
{ "answer": "<English answer>", "computed_fields": { "numeric values used" } }
"""

def ask_gpt(question_en, csv_block):
    user = f"CSV data:\n{csv_block}\n\nQUESTION:\n{question_en}"
    resp = client.chat.completions.create(
        model="gpt-4o",
        temperature=0,
        messages=[{"role": "system", "content": SYSTEM}, {"role": "user", "content": user}]
    ).choices[0].message.content
    clean = re.sub(r"^```json|```$", "", resp.strip(), flags=re.M | re.I)
    start, end = clean.find("{"), clean.rfind("}")
    return json.loads(clean[start:end+1])

def translate_to_spanish(english_text):
    prompt = f"Translate this English medical answer into clear, neutral Spanish:\n{english_text}"
    return client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    ).choices[0].message.content.strip()

def text_to_speech_spanish(text, filename, voice="alloy"):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with client.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts", voice=voice, input=text
    ) as response:
        response.stream_to_file(filename)
    print(f"🔊 Saved Spanish audio: {filename}")
    return filename


# ================================================================
# 4️⃣ MAIN PIPELINE
# ================================================================
def run_full_pipeline(csv_path, audio_folder, audio_files):
    # Step 1 — Transcribe and Translate Spanish Audio
    trans_csv = os.path.join(CSV_OUTPUT_FOLDER, "audio_translations.csv")
    trans_df = process_and_translate_audio(audio_folder, audio_files, trans_csv)

    # Step 2 — Evaluate ASR (WER, CER, SER)
    gt_csv = os.path.join(audio_folder, "ground_truth.csv")
    asr_csv = os.path.join(CSV_OUTPUT_FOLDER, "asr_metrics.csv")
    asr_df = evaluate_asr_performance(gt_csv, trans_csv, asr_csv)

    # Step 3 — Load Blood Pressure Data
    df_bp = pd.read_csv(csv_path)
    csv_block = df_bp.to_csv(index=False)

    results = []
    for i, row in trans_df.iterrows():
        q_num = i + 1
        q_en = row["english_translation"]
        print(f"\n🔹 Q{q_num}: {q_en}")

        try:
            ans = ask_gpt(q_en, csv_block)
            ans_en = ans.get("answer", "").strip()
            ans_es = translate_to_spanish(ans_en)

            audio_file = os.path.join(AUDIO_OUTPUT_FOLDER, f"answer_{q_num}_es.wav")
            text_to_speech_spanish(ans_es, audio_file)

            results.append({
                "question_number": q_num,
                "audio_file_in": row["audio_file"],
                "spanish_question": row["spanish_transcription"],
                "english_question": q_en,
                "english_answer": ans_en,
                "spanish_answer": ans_es,
                "audio_answer_file": audio_file,
                "computed_fields": json.dumps(ans.get("computed_fields", {}))
            })
            print(f"✅ Completed Q{q_num}")

        except Exception as e:
            print(f"❌ Error Q{q_num}: {e}")

    # Step 4 — Save Final Results
    final_csv = os.path.join(CSV_OUTPUT_FOLDER, "final_pipeline_results.csv")
    pd.DataFrame(results).to_csv(final_csv, index=False)
    print(f"\n✅ All results saved to {final_csv}")
    return results


# ================================================================
# 5️⃣ RUN
# ================================================================
csv_path = os.path.join(BASE_PATH, "Data", "synthetic_bp_one_person.csv")
run_full_pipeline(csv_path, AUDIO_INPUT_FOLDER, audio_files)






✅ Found 4 audio file(s): ['q2_es.wav', 'q3_es.wav', 'q4_es.wav', 'q1_es.wav']


100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 62.9MiB/s]



🎯 STARTING SPANISH TRANSCRIPTION + TRANSLATION
🎧 Transcribing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q2_es.wav


100%|██████████| 312/312 [00:04<00:00, 66.77frames/s]



[1] q2_es.wav
🇪🇸 ¿Cuáles fueron los valores de la última semana?
🇬🇧 What were the values from the last week?
🎧 Transcribing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q3_es.wav


...

100%|██████████| 398/398 [00:02<00:00, 147.75frames/s]



[3] q4_es.wav
🇪🇸 ¿Cuáles son los rango normales para una persona como yo?
🇬🇧 What are the normal ranges for a person like me?
🎧 Transcribing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q1_es.wav


100%|██████████| 324/324 [00:04<00:00, 66.42frames/s]



[4] q1_es.wav
🇪🇸 ¿Cuáles son mis valores de presión arterial hoy?
🇬🇧 What are my blood pressure readings today?

✅ Transcriptions + translations saved to /content/drive/MyDrive/health-tequity-case/data/csv_results/audio_translations.csv


FileNotFoundError: ❌ Missing GT file: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/ground_truth.csv

In [20]:
   !pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 6.0 MB/s eta 0:00:00


# **Define Templates + GPT call (JSON output)**

In [23]:
# import json, re
# from openai import OpenAI

# client = OpenAI(api_key=api_key)

# TEMPLATES = {
#     "today":         "Your systolic blood pressure was {sys} mm of Hg and your diastolic blood pressure was {dia} mm of Hg.",
#     "last_week":     "Over the last week, your systolic blood pressure has averaged {sys_avg} mm of Hg and your diastolic blood pressure has averaged {dia_avg} mm of Hg.",
#     "trend_month":   "The trend for the values over the last month has been {trend} average values of your systolic blood pressure and diastolic blood pressure.",
#     "normal_ranges": "While each person’s normal range should be discussed with their physician, literature suggests that for a {sex} aged {age} years, systolic and diastolic blood pressure can typically be expected to be {sys_norm} mm Hg and {dia_norm} mm Hg respectively. This information was retrieved from {reference}."
# }

# SYSTEM = """You are a careful data analyst.
# Do ALL analysis yourself using ONLY the CSV provided by the user.
# Interpret columns: date, age, sex, systolic, diastolic.
# - "Today" = most recent row by date.
# - "Last week" = last 7 rows by date (including the most recent).
# - "Trend over the last month" = last 30 rows; return one of: increasing / decreasing / stable.
# - If a specific date is mentioned (e.g., 'on October 1'), return that date’s values if present.
# Return STRICT JSON ONLY:
# {
#  "template": "today"|"last_week"|"trend_month"|"normal_ranges",
#  "fields": {...},   # only the slots the chosen template needs (e.g., sys, dia, sys_avg, dia_avg, trend, age, sex, sys_norm, dia_norm, reference, date)
#  "final_text": "one sentence exactly following the chosen template with mm of Hg units"
# }
# No extra prose. JSON only.
# """

# def ask_gpt(question_en: str, csv_block: str) -> dict:
#     user = f"CSV:\n{csv_block}\n\nQUESTION:\n{question_en}\n\nReturn JSON only."
#     resp = client.chat.completions.create(
#         model="gpt-4o",     # or "gpt-4o-mini" for lower cost
#         temperature=0,
#         messages=[
#             {"role":"system","content":SYSTEM},
#             {"role":"system","content":"Templates:\n" + json.dumps(TEMPLATES)},
#             {"role":"user","content":user}
#         ]
#     ).choices[0].message.content

#     # Strip possible code fences and parse only the JSON blob
#     clean = re.sub(r"^```json|```$", "", resp.strip(), flags=re.M|re.I)
#     start, end = clean.find("{"), clean.rfind("}")
#     return json.loads(clean[start:end+1])



# **Run the 4 required question and save results**

In [5]:
# import pandas as pd, os

# questions = [
#     "What are my systolic and diastolic blood pressures today?",
#     "What were the values over the last week?",
#     "What is the trend of the values over the last month?",
#     "What are the normal ranges for a person like me?",
#     # Optional extension:
#     "What were my blood pressure values on October 1?"
# ]

# rows = []
# for q in questions:
#     obj = ask_gpt(q, csv_text)
#     rows.append({
#         "question": q,
#         "template": obj.get("template"),
#         "fields": json.dumps(obj.get("fields", {}), ensure_ascii=False),
#         "answer_en": obj.get("final_text")
#     })
#     print(f"\nQ: {q}\nA: {obj.get('final_text')}")

# # os.makedirs("/content/drive/MyDrive/health-tequity-case", exist_ok=True)
# out_path = "/content/drive/MyDrive/health-tequity-case/Data/llm_answers_en_colab.csv"
# pd.DataFrame(rows).to_csv(out_path, index=False, encoding="utf-8-sig")
# print(f"\n✅ Saved: {out_path}")


TEXT-to-Speech(**TTS**)

In [2]:
# !pip install gTTS


In [1]:
# from gtts import gTTS
# import os

# # Spanish text (your model’s translated output)
# spanish_answer = "Su presión arterial sistólica fue de 135 milímetros de mercurio y su presión diastólica fue de 88."

# # Create TTS object (language code 'es' = Spanish)
# tts = gTTS(text=spanish_answer, lang='es')

# # Save the output file
# output_path = "data/output_q1_es.wav"
# tts.save(output_path)

# print(f"✅ Spanish audio saved: {output_path}")
